In [1]:
%load_ext autoreload

In [2]:
import torch
import torchvision.datasets as datasets
import torchvision
from pathlib import Path
import sys
sys.path.append(str(Path('../../').resolve()))

In [3]:
%autoreload
import v4.pytorch_unet.unet as unet
from v4.pytorch_unet.learner import Learner

In [4]:
mnist = datasets.MNIST(Path('./mnist'), download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))
train_loader = torch.utils.data.DataLoader(mnist, batch_size=4000, shuffle=True, num_workers=4, pin_memory=True)

In [5]:
model = unet.ResNetUnet(in_channels=1, in_size=list(mnist.data[0].shape), n_classes=len(mnist.classes), depth=2)
#model.cuda();

In [6]:
model(torch.rand(1, 1, 256, 256))

DOWN OUT SHAPE [28, 28] 0 torch.Size([1, 64, 128, 128])
DOWN OUT SHAPE (14, 14) 1 torch.Size([1, 128, 64, 64])
UP! CURRENT X torch.Size([1, 128, 64, 64]) SHORTCUT? torch.Size([1, 128, 64, 64])
BYPASS! torch.Size([1, 128, 32, 32]) XX torch.Size([1, 64, 128, 128])


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 32 and 128 in dimension 2 at ../aten/src/TH/generic/THTensor.cpp:711

In [19]:
learner = Learner(model, train_loader, valid_loader=None, cuda=True)

In [29]:
learner.train(n_epochs=2, lr=3e-3, momentum=0.9)

Epoch 18 train loss: 0.003327038697898388 valid loss: 0
Epoch 19 train loss: 0.0029454907247175774 valid loss: 0
